# **Solar Data Analysis with SunPy**

![sunpy logo](./images/sunpy_logo.png)

## SPHERE Workshop / 11 July 2022

### *Will Barnes* (AU/NASA GSFC) *on behalf of the SunPy Community*
### *with many thanks to: Albert Shih (NASA GSFC), Stuart Mumford (Aperio Software), Laura Hayes (ESA ESTEC), David Stansby (UCL)*

## What is SunPy?

![sunpy core summary](./images/sunpy-summary-slide.png)

![sunpy ecosystem summary](./images/sunpy-ecosystem-slide.png)

In this tutorial, we will be looking at observations from a flare that occurred on [February 15 2011](https://hesperia.gsfc.nasa.gov/rhessi_extras/flare_images/2011/02/15/20110215_0127_0227/hsi_20110215_0127_0227.html). We'll divide this into three sections:

1. Searching for and Downloading Data
2. Loading Data into `Map` and `TimeSeries`
3. Combining Data through Reprojection

### Additional Resources

Below are a few helpful links for finding out more about SunPy, including our webpage, documentation, and our Matrix channel:

* [sunpy.org](https://sunpy.org/)
* [Example Gallery](https://docs.sunpy.org/en/stable/generated/gallery/index.html)
* [Installation Instructions](https://docs.sunpy.org/en/stable/installation.html)
* [Tutorial Notebooks from 2021 AAS/SPD SunPy Workshop](https://github.com/sunpy/aas-2021-workshop)
* [Tutorial Notebooks from 2022 PyHC Summer School](https://github.com/heliophysicsPy/summer-school/tree/main/sunpy-tutorial)
* [List of Affiliated Packages](https://sunpy.org/project/affiliated.html)
* [Matrix Chat](https://openastronomy.element.io/#/room/#sunpy:openastronomy.org)
* [OpenAstronomy Discourse](https://community.openastronomy.org/c/sunpy/5)

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import ImageNormalize, AsymmetricPercentileInterval
import astropy.wcs
from reproject.mosaicking import reproject_and_coadd
from reproject import reproject_interp
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

import sunpy.map
import sunpy.timeseries
from sunpy.net import Fido, attrs as a
from sunkit_instruments.rhessi import imagecube2map

## 1. Searching for and Downloading Data

* `Fido`is a unified interface for searching and downloading solar physics data.
* Search and accesses multiple instruments and all available data providers in a single query.
* In `sunpy`, `Fido` offers access to data available through: VSO, JSOC, HEK, HELIO, individual data providers from web accessible sources (http, ftp, etc)
* `clients` defined in `sunpy` or in other packages provide access to many sources of data

In [ ]:
Fido

In [ ]:
a.Instrument

Define our search interval to be consistent with the interval of the RHESSI image cube

In [ ]:
flare_start = "2011-02-15T01:27"
flare_end = "2011-02-15T02:27"

The `near=...` keyword chooses a single observation in a given interval closest to a given time

In [ ]:
flare_time = a.Time(flare_start, flare_end, near='2011-02-15T01:55')

In [ ]:
aia_query = flare_time & a.Wavelength(193*u.angstrom) & a.Instrument.aia

In [ ]:
Fido.search(aia_query)

We can also logically combine queries for multiple instruments at once.
What if we also wanted to look for the STEREO and GOES XRS data during this same interval?

In [ ]:
aia_query = a.Instrument.aia & a.Wavelength(193*u.angstrom)
secchi_query = a.Instrument.secchi & (a.Source('STEREO_A') | a.Source('STEREO_B')) & a.Wavelength(195*u.Angstrom)

In [ ]:
goes_query = a.Instrument.xrs & a.goes.SatelliteNumber(15)

In [ ]:
combined_query = Fido.search(flare_time, aia_query | secchi_query | goes_query )

In [ ]:
combined_query

We can easily make a single download request for all of our data by passing in our combined query for AIA, STEREO, and XRS.

In [ ]:
files = Fido.fetch(combined_query, path='data/{source}/{instrument}')

In [ ]:
files

## 2. Data Structures

### Loading Image Data with `Map`

We create a `sunpy.map.Map` object by passing in the FITS file for a single AIA, STEREO A, and STEREO B.

In [ ]:
m_aia = sunpy.map.Map('data/SDO/AIA/aia_lev1_193a_2011_02_15t01_54_57_63z_image_lev1.fits')
m_stereo_a = sunpy.map.Map('data/STEREO_A/SECCHI/20110215_015530_n4eua.fts')
m_stereo_b = sunpy.map.Map('data/STEREO_B/SECCHI/20110215_015530_n4eub.fts')

We can easily visualize a map after loading it using the quicklook functionality.

In [ ]:
m_stereo_a

### Attributes of `Map`

`Map` provides a common interface to most 2D imaging solar datasets and provides several useful pieces of metadata.
`Map` is a container for holding your data and metadata (usually from the FITS header) together.

The `.meta` and `.data` attributes provide access to the metadata and underlying array of image data, respectively.

In [ ]:
m_aia.data

In [ ]:
m_aia.meta

However, this metadata can be terse, non-homogeneous, and sometimes difficult to parse.
`Map` provides several attributes derived from the underlying raw metadata that expose a uniform interface to the metadata for each map.

In [ ]:
m_aia.date

In [ ]:
m_stereo_a.date

In [ ]:
m_aia.wavelength

In [ ]:
m_aia.rsun_obs

In [ ]:
m_stereo_a.rsun_obs

In [ ]:
m_aia.processing_level

In [ ]:
m_stereo_a.instrument

Each `Map` object also holds the unit system that the image data is in, expressed in terms of an `astropy.unit.Unit` object.

In [ ]:
m_stereo_a.unit

### Coordinate Information

Each `Map` also exposes information about which coordinate system the image was taken in, including the location of the spacecraft that recorded that observation.

`sunpy` leverages and extends the powerful `astropy` coordinate framework.

For each `Map`, we can easily access what *coordinate frame* the observation cooresponds to.

In [ ]:
m_aia.coordinate_frame

Similarly, we can look at the location of the observer (as defined by the position of the satellite at the time of the observation).

In [ ]:
m_aia.observer_coordinate

In [ ]:
m_stereo_a.observer_coordinate

In [ ]:
m_stereo_b.observer_coordinate

We can plot these observer coordinates to show the relative position, in heliographic longitude, of each spacecraft.
Additionally, the [SolarMACH tool](https://solar-mach.github.io/) provides a web interface for building similar visualizations.

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(projection='polar')

# Plot the Sun
ax.plot(0, 0, marker='o', markersize=20, label='Sun', color='yellow')

# Plot the satellite locations
for m in [m_aia, m_stereo_a, m_stereo_b]:
    sat = m.observatory
    coord = m.observer_coordinate
    ax.plot(coord.lon.to('rad'), coord.radius.to(u.AU), 'o', label=sat)

ax.set_theta_zero_location("S")
ax.set_rlabel_position(90)
ax.set_rlim(0, 1.3)
ax.legend()

We can also use this coordinate information to overplot the flare position in each map.
We can easily locate the approximate flare position in the AIA image.

In [ ]:
m_aia.plot()

In [ ]:
flare_location = SkyCoord(Tx=200*u.arcsec, Ty=-220*u.arcsec, frame=m_aia.coordinate_frame)

And transform it to the coordinate frame of either STEREO satellite

In [ ]:
flare_location.transform_to(m_stereo_a.coordinate_frame)

In [ ]:
flare_location.transform_to(m_stereo_b.coordinate_frame)

Note that we can use the `plot_coord` command to overlay this coordinate on each `Map` and perform this transformation automatically before plotting

In [ ]:
for i, m in enumerate([m_aia, m_stereo_a,]):
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111, projection=m)
    im = m.plot(axes=ax, clip_interval=(10,99.9)*u.percent)
    # Manually plot flare location
    ax.plot_coord(flare_location, ls=' ', marker='x', color='C3', markersize=20)

### Visualization

`Map` provides some additional "helpers" for plotting the associated image data with the correct projection based on the WCS.
At a minimum, this can be accomplished through the `.plot()` method.

In [ ]:
m_aia.plot()
plt.colorbar()

This "automagically" creates a figure and an axis (with a projection based on the WCS of the map) and plots our map on that axis, with a colormap and normalization tailored for the specific map source.
All of this visualization is built on top of `matplotlib` and the `WCSAxes` capabilities provided by `astropy` that we saw demoed earlier today.
However, as you can see, the resulting default colorbar is not particularly useful.

Because all of this plotting capability is built on top of `matplotlib`, we can easily customize the various components of our plot.

We can explictly create the axis and add the projection for the map if we want more fine-grained control over labels and titles.
We can also easily adjust the limits on our colorbar using the `clip_interval` key.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection=m_aia)
im = m_aia.plot(axes=ax, clip_interval=(1,99.9)*u.percent)
m_aia.draw_grid(axes=ax, lw=1, alpha=1)
ax.set_title(r'A nicer AIA 193 $\mathrm{\AA}$ Plot')
ax.coords[0].set_axislabel('HPC Lon')
ax.coords[1].set_axislabel('HPC Lat')
fig.colorbar(im)

Using `matplotlib` combined with `WCSAxes`, we can build more complex, publication-quality visualizations.

(**NOTE:** It is not necessary to fully understand every intricacy of the plotting code below during the course of the tutorial. This is merely to show how `Map.plot` can be be used to make more complex plots.)

In [ ]:
fig = plt.figure(figsize=(15,5))

for i, m in enumerate([m_aia, m_stereo_a, m_stereo_b]):
    # Create the axis with the appropriate projection
    ax = fig.add_subplot(1, 3, i+1, projection=m)
    
    # Add the plot to the axis
    im = m.plot(axes=ax, annotate=False, clip_interval=(10,99.9)*u.percent)
    
    # Make the HPC grid lines visible
    ax.coords.grid(alpha=1, ls='-')
    
    # Adjust the labels and ticks
    if i > 0:
        ax.coords[1].set_auto_axislabel(False)
    else:
        ax.coords[1].set_axislabel('Solar-Y')
    ax.coords[0].set_axislabel('Solar-X')
    ax.coords[1].set_ticklabel(rotation=90,)
    
    # Put a label on each plot
    ax.text(m.data.shape[1]//2, m.data.shape[0]*.97, m.observatory,
            color='w',
            horizontalalignment='center',
            verticalalignment='top',
            fontsize=14)
    
    # Add a colorbar to the top of each plot
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('top', size='4%', pad=0.1, axes_class=matplotlib.axes.Axes)
    cb = fig.colorbar(im, cax=cax, orientation='horizontal')
    cax.xaxis.set_ticks_position("top")
    cax.xaxis.set_tick_params(direction='in',)
    cb.locator = matplotlib.ticker.MaxNLocator(nbins=3)
    cb.update_ticks()
    
plt.subplots_adjust(wspace=0.1)

### Animating RHESSI Image Cubes with `MapSequence`

In addition, the `MapSequence` container provides a data container for holding multiple maps, such as in the case where you have a sequence of maps taken at successive times.

Let's download the CLEAN image cube for this particular flare from the RHESSI flare archive: https://hesperia.gsfc.nasa.gov/rhessi_extras/flare_images/2011/02/15/20110215_0127_0227/hsi_20110215_0127_0227.html.
We can then use the [`sunkit-instruments`](https://docs.sunpy.org/projects/sunkit-instruments/en/stable/index.html) package to load this data into a `MapSequence`.

In [ ]:
rhessi_cube = imagecube2map('data/hsi_imagecube_clean_20110215_0127_41tx5e.fits')

In [ ]:
rhessi_cube.keys()

The `MapSequence` can be indexed to return the individual `Map` objects at each time step.

In [ ]:
rhessi_cube['6-12 keV'][20].peek()

One of the most useful features of a `MapSequence` is the ability to create coordinate-aware visualizations of our stack of `Map` objects.
To do this, we'll first create a a colormap normalization appropriate to the range of the data for every map in our stack.

In [ ]:
vmax = np.max(rhessi_cube['25-50 keV'].as_array())
norm=ImageNormalize(vmin=0, vmax=vmax) 

The `plot` method on our `MapSequence` object now returns an animation rather than a simple static plot.

In [ ]:
plt.figure(figsize=(10,10))
rhessi_ani = rhessi_cube['25-50 keV'].plot(norm=norm)

In [ ]:
HTML(rhessi_ani.to_jshtml())

# Rotating Images

The `.rotate` method applies a rotation in the image plane, i.e. about an axis out of the page. 
In the case where we do not specify an angle (or rotation matrix), the image will be rotated such that the world and pixel axes are aligned.
In the case of an image in helioprojective coordinate system, this means that solar north will be aligned with the y-like pixel axis of the image.

In [ ]:
m_stereo_a_rot = m_stereo_a.rotate()

In [ ]:
fig = plt.figure(figsize=(11,5))
ax = fig.add_subplot(121,projection=m_stereo_a)
m_stereo_a.plot(axes=ax, vmin=800, vmax=5e3)
ax.coords.grid(alpha=1, ls='-')
ax = fig.add_subplot(122,projection=m_stereo_a_rot)
m_stereo_a_rot.plot(axes=ax, vmin=800, vmax=5e3)
ax.coords.grid(alpha=1, ls='-')

This rotation is also reflected in the updated metadata of the rotated image.

In [ ]:
m_stereo_a.rotation_matrix

In [ ]:
m_stereo_a_rot.rotation_matrix

### Cropping Images with `submap`

We commonly want to pare down our full field-of-view to a particular region of interest.
With a map, we can do this using the `submap` method.

In [ ]:
m_stereo_a_rot_cropped = m_stereo_a_rot.submap(SkyCoord(-1500*u.arcsec, -1500*u.arcsec, frame=m_stereo_a_rot.coordinate_frame),
                                               width=3000*u.arcsec, height=3000*u.arcsec)

In [ ]:
m_stereo_a_rot_cropped.plot(vmin=800, vmax=5e3)

Additionally, we can use the corners of our RHESSI maps to crop our full-disk AIA image to the FOV that RHESSI observed.

In [ ]:
m_aia_cropped = m_aia.submap(rhessi_cube['6-12 keV'][0].bottom_left_coord,
                             top_right=rhessi_cube['6-12 keV'][0].top_right_coord)

In [ ]:
m_aia_cropped.plot()

We can then lay the contours from one of our RHESSI images on top of our cropped AIA data.

In [ ]:
rhessi_cube['6-12 keV'][19].date

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection=m_aia_cropped)
m_aia_cropped.plot(axes=ax)
rhessi_cube['6-12 keV'][19].draw_contours(np.arange(5,100,5)*u.percent, axes=ax, alpha=0.75, cmap='viridis')

## The `Timeseries` Data Structure

In addition to `Map` for 2D image data, `sunpy` also provides a container for tabular time series data through the `TimeSeries` class.
We can create a `TimeSeries` object in a very similar manner to how we create a `Map` object.

Let's look at the corresponding GOES XRS data that we downloaded in the previous notebook.

In [ ]:
ts = sunpy.timeseries.TimeSeries('data/GOES/XRS/sci_gxrs-l2-irrad_g15_d20110215_v0-0-0.nc')

In [ ]:
ts

As with `Map`, `TimeSeries` acts as a container for the data + metadata. We can access each component individually.

In [ ]:
ts.meta

The `TimeSeries` object can also be converted to other formats like a `pandas` `DataFrame`

In [ ]:
ts.to_dataframe()

There are also a number of attributes on each `TimeSeries` derived from the data/metadata.

In [ ]:
ts.columns

In [ ]:
ts.observatory

In [ ]:
ts.source

In [ ]:
ts.units

### Slicing and Visualizing `TimeSeries`

Note that this intensity `TimeSeries` spans 24 h of observation time and recall that we are only interested in the ~1 h interval corresponding to the flare.
We can truncate our timeseries around the times of interest.

In [ ]:
ts_flare = ts.truncate(flare_start, flare_end)

And then do a quicklook on our lightcurve.

In [ ]:
ts_flare

## Combining Image Data from Multiple Observers

When combining these images all three need to assume the same radius of
the Sun for the data. The AIA images specify a slightly different value
than the IAU 2015 constant. To avoid coordinate transformation issues we
reset this here.



In [ ]:
m_stereo_a.meta['rsun_ref'] = sunpy.sun.constants.radius.to_value(u.m)
m_stereo_b.meta['rsun_ref'] = sunpy.sun.constants.radius.to_value(u.m)
m_aia.meta['rsun_ref'] = sunpy.sun.constants.radius.to_value(u.m)

The next step is to calculate the output coordinate system for the combined
map. We select a heliographic Stonyhurst frame, and a Plate Carree (CAR)
projection, and generate a header using `sunpy.map.make_fitswcs_header` and
then construct a World Coordinate System (WCS) object for that header.



In [ ]:
shape_out = (360, 720)  # This is set deliberately low to reduce memory consumption
reference_coordinate = SkyCoord(0, 0, unit=u.deg, frame="heliographic_stonyhurst", obstime=m_aia.date)
scale = [360 / shape_out[1], 180 / shape_out[0]] * u.deg / u.pix

In [ ]:
header = sunpy.map.make_fitswcs_header(shape_out,
                                       reference_coordinate,
                                       scale=scale,
                                       wavelength=m_aia.wavelength,
                                       projection_code="CAR")
out_wcs = astropy.wcs.WCS(header)

Next we call the `reproject.mosaicking.reproject_and_coadd` function, which
takes a list of maps, and the desired output WCS and array shape.



In [ ]:
maps = [m_aia, m_stereo_a, m_stereo_b]

In [ ]:
array,_ = reproject_and_coadd(maps, out_wcs, shape_out, reproject_function=reproject_interp)

In [ ]:
combined_map = sunpy.map.Map(array, header)

In [ ]:
plt.figure(figsize=(10,5))
combined_map.plot(cmap='euvi195', norm=m_aia.plot_settings['norm'])

As you can see this leaves a little to be desired. To reduce the obvious warping towards the points which are close to the limb in the input images, we can define a set of weights to use when co-adding the output arrays.
To reduce this warping we want to calculate an set of weights which highly weight points close to the center of the disk in the input image.

We can achieve this by using sunpy's coordinate framework. First we calculate all the world coordinates for all the pixels in all three input maps.

In [ ]:
coordinates = tuple(map(sunpy.map.all_coordinates_from_map, maps))

To get a weighting which is high close to disk centre and low towards
the limb, we can use the Z coordinate in the heliocentric frame. This
coordinate is the distance of the sphere from the centre of the Sun
towards the observer.



In [ ]:
weights = [coord.transform_to("heliocentric").z.value for coord in coordinates]

These weights are good, but they are better if the ramp down is a little
smoother, and more biased to the centre. Also we can scale them to the
range 0-1, and set any off disk (NaN) regions to 0.



In [ ]:
weights = [(w / np.nanmax(w)) ** 1.5 for w in weights]
for w in weights:
    w[np.isnan(w)] = 0

In [ ]:
plt.imshow(weights[0])
plt.colorbar()

Now we can rerun the reprojection. This time we also set
``match_background=True`` which scales the images by a single scaling
factor so they are of similar brightness. We also set
``background_reference=0`` which uses the AIA map as the reference for
the background scaling.

In [ ]:
array, _ = reproject_and_coadd(maps, out_wcs, shape_out,
                               input_weights=weights,
                               reproject_function=reproject_interp,
                               match_background=True,
                               background_reference=0)

Once again we create a new map from our reprojected array and new header.

In [ ]:
combined_map = sunpy.map.Map(array, header)

In [ ]:
fig = plt.figure(figsize=(15, 6))
ax = fig.add_subplot(projection=combined_map)
im = combined_map.plot(axes=ax, cmap='euvi195', norm=m_aia.plot_settings['norm'],
                       clip_interval=(1,99.5)*u.percent,
                       title='AIA, STEREO A and B')

# Fix the tick formatting
lon, lat = ax.coords
lon.set_coord_type("longitude")
lon.coord_wrap = 180
lon.set_format_unit(u.deg)
lat.set_coord_type("latitude")
lat.set_format_unit(u.deg)

# Fix the labeling
lon.set_axislabel('Heliographic Longitude', minpad=0.8)
lat.set_axislabel('Heliographic Latitude', minpad=0.9)
lon.set_ticks(spacing=25*u.deg, color='k')
lat.set_ticks(spacing=15*u.deg, color='k')

# Reset the view to pixel centers
_ = ax.axis((0, shape_out[1], 0, shape_out[0]))

# Draw the limb as defined by each spacecraft
m_aia.draw_limb(axes=ax, lw=2, color='C0', label='AIA')
m_stereo_a.draw_limb(axes=ax, lw=2, color='C1', label='STEREO A')
m_stereo_b.draw_limb(axes=ax, lw=2, color='C4', label='STEREO B')

# Plot approximate flare position
ax.plot_coord(flare_location, ls=' ', marker='x', color='C3', markersize=20)
    
# Add the legend and colorbar
plt.legend(loc=2)
plt.colorbar(im, ax=ax)